## Step 1: Configure Google Cloud Authentication

For Vertex AI models, you need to authenticate with Google Cloud.

In [ ]:
import os

# TODO(developer): Replace with your Google Cloud Project ID
project_id = "your-project-id"  # e.g., "my-dnb-project"

# Set the project for gcloud (if running locally)
!gcloud config set project {project_id}

# Verify authentication
!gcloud auth application-default login

## Step 2: Connect to Toolbox Server

Verify the Toolbox server is running and accessible.

In [ ]:
# In the cell that sets up the agent and runner, add this at the very beginning:

import os

# Configure for Vertex AI
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

# TODO(developer): Replace with your Google Cloud Project ID and location
project_id = "your-project-id"  # e.g., "my-dnb-project"
location = "us-central1"  # or your preferred region

# Initialize Vertex AI
import vertexai
vertexai.init(project=project_id, location=location)

# Now import ADK components
from google.adk import Agent
from google.adk.runners import Runner
from google.adk.tools.toolbox_toolset import ToolboxToolset
from google.adk.sessions import InMemorySessionService
from google.adk.artifacts.in_memory_artifact_service import (
    InMemoryArtifactService,
)
from google.genai import types
import requests

# ...existing code...

TOOLBOX_URL = "http://localhost:5000"

try:
  # Get available toolsets
  response = requests.get(f"{TOOLBOX_URL}/api/toolsets")
  if response.status_code == 200:
    toolsets = response.json()
    print("Available toolsets:", list(toolsets.keys()))

  # Get available tools
  response = requests.get(f"{TOOLBOX_URL}/api/tools")
  if response.status_code == 200:
    tools = response.json()
    print("Available tools:", list(tools.keys()))
except Exception as e:
  print(f"Error discovering tools: {e}")

# Create agent with Toolbox tools using ADK's native integration
agent = Agent(
  name="dnb_toolbox_agent",
  model="gemini-2.0-flash",
  instruction="You are a helpful assistant that can access DNB APIs.",
  tools=[
    ToolboxToolset(
      server_url="http://localhost:5000",
      toolset_name="dnb_echo_tools"
    )
  ]
)

# Create services and runner
app_name = "dnb_toolbox_app"
user_id = "user1"
session_service = InMemorySessionService()
artifact_service = InMemoryArtifactService()

runner = Runner(
  app_name=app_name,
  agent=agent,
  session_service=session_service,
  artifact_service=artifact_service,
)

# Create a session (using await since it's async)
session = await session_service.create_session(
  app_name=app_name,
  user_id=user_id
)

# Create the message content using ADK types
content = types.Content(
  role='user',
  parts=[types.Part(text="What's in the DNB API?")]
)

# Run the agent with proper parameters
async for event in runner.run_async(
  user_id=user_id,
  session_id=session.id,
  new_message=content
):
  if event.content and event.content.parts:
    if text := ''.join(part.text or '' for part in event.content.parts):
      print(f'[{event.author}]: {text}')